In [1]:
import almasim.astro as uas
import numpy as np 
import pandas as pd 
import astropy.units as U
import almasim.alma as ual
import os
from astropy.constants import c
from astropy.cosmology import FlatLambdaCDM
import math
from math import pi, ceil


In [2]:
def remove_non_numeric(text):
        """Removes non-numeric characters from a string.
        Args:
            text: The string to process.

        Returns:
            A new string containing only numeric characters and the decimal point (.).
        """
        numbers = "0123456789."
        return "".join(char for char in text if char in numbers)

def freq_supp_extractor(freq_sup, obs_freq):
        freq_band, n_channels, freq_mins, freq_maxs, freq_ds = [], [], [], [], []
        freq_sup = freq_sup.split("U")
        for i in range(len(freq_sup)):
            sup = freq_sup[i][1:-1].split(",")
            sup = [su.split("..") for su in sup][:2]
            freq_min, freq_max = float(remove_non_numeric(sup[0][0])), float(
                remove_non_numeric(sup[0][1])
            )
            freq_d = float(remove_non_numeric(sup[1][0]))
            freq_min = freq_min * U.GHz
            freq_max = freq_max * U.GHz
            freq_d = freq_d * U.kHz
            freq_d = freq_d.to(U.GHz)
            freq_b = freq_max - freq_min
            n_chan = int(freq_b / freq_d)
            freq_band.append(freq_b)
            n_channels.append(n_chan)
            freq_mins.append(freq_min)
            freq_maxs.append(freq_max)
            freq_ds.append(freq_d)
        freq_ranges = np.array(
            [[freq_mins[i].value, freq_maxs[i].value] for i in range(len(freq_mins))]
        )
        idx_ = np.argwhere(
            (obs_freq.value >= freq_ranges[:, 0])
            & (obs_freq.value <= freq_ranges[:, 1])
        )[0][0]
        freq_range = freq_ranges[idx_]
        band_range = freq_range[1] - freq_range[0]
        n_channels = n_channels[idx_]
        central_freq = freq_range[0] + band_range / 2
        freq_d = freq_ds[idx_]
        return band_range * U.GHz, central_freq * U.GHz, n_channels, freq_d

def cont_finder(cont_frequencies, line_frequency):
        # cont_frequencies=sed['GHz'].values
        distances = np.abs(
            cont_frequencies - np.ones(len(cont_frequencies)) * line_frequency
        )
        return np.argmin(distances)

def normalize_sed(
    sed,
    lum_infrared,
    solid_angle,
    cont_sens,
    freq_min,
    freq_max,
    remote=False,
):
    so_to_erg_s = 3.846e33  # Solar luminosity to erg/s -XX
    lum_infrared_erg_s = lum_infrared * so_to_erg_s  # luminosity in erg/s -XX
    sed["Jy"] = lum_infrared_erg_s * sed["erg/s/Hz"] * 1e23 / solid_angle
    cont_mask = (sed["GHz"].values >= freq_min) & (sed["GHz"].values <= freq_max)
    if sum(cont_mask) > 0:
        cont_fluxes = sed["Jy"].values[cont_mask]
        min_ = np.min(cont_fluxes)
    else:
        freq_point = np.argmin(np.abs(sed["GHz"].values - freq_min))
        cont_fluxes = sed["Jy"].values[freq_point]
        min_ = cont_fluxes
    lum_save = lum_infrared
    while min_ > cont_sens:
        lum_infrared -= 0.1 * lum_infrared
        lum_infrared_erg_s = so_to_erg_s * lum_infrared
        sed["Jy"] = lum_infrared_erg_s * sed["erg/s/Hz"] * 1e23 / solid_angle
        cont_mask = (sed["GHz"] >= freq_min) & (sed["GHz"] <= freq_max)
        if sum(cont_mask) > 0:
            cont_fluxes = sed["Jy"].values[cont_mask]
            min_ = np.min(cont_fluxes)
        else:
            freq_point = np.argmin(np.abs(sed["GHz"].values - freq_min))
            cont_fluxes = sed["Jy"].values[freq_point]
            min_ = cont_fluxes

    return sed, lum_infrared_erg_s, lum_infrared
def sed_reading(
    type_,
    path,
    cont_sens,
    freq_min,
    freq_max,
    remote,
    lum_infrared=None,
    redshift=None,
):
    cosmo = FlatLambdaCDM(H0=70 * U.km / U.s / U.Mpc, Tcmb0=2.725 * U.K, Om0=0.3)
    if (
        type_ == "extended"
        or type_ == "diffuse"
        or type_ == "molecular"
        or type_ == "galaxy-zoo"
        or type_ == "hubble-100"
    ):
        file_path = os.path.join(path, "SED_low_z_warm_star_forming_galaxy.dat")
        if redshift is None:
            redshift = 10 ** (-4)
        if lum_infrared is None:
            lum_infrared = 1e12  # luminosity in solar luminosities
    elif type_ == "point" or type_ == "gaussian":
        file_path = os.path.join(path, "SED_low_z_type2_AGN.dat")
        if redshift is None:
            redshift = 0.05
        if lum_infrared is None:
            lum_infrared = 1e12  # luminosity in solar luminosities
    else:
        return "Not valid type"
    # L (erg/s/Hz) = 4 pi d^2(cm) * 10^-23 Flux (Jy)
    #  Flux (Jy) =L (erg/s/Hz) * 10^23 /  * 4 pi d^2(cm)
    # To normalize we multiply by lum_infrared_jy
    distance_Mpc = cosmo.luminosity_distance(redshift).value  # distance in Mpc
    Mpc_to_cm = 3.086e24  # Mpc to cm
    distance_cm = distance_Mpc * Mpc_to_cm  # distance in cm  -XX
    solid_angle = 4 * pi * distance_cm**2  # solid angle in cm^2 -XX
    # Load the SED
    sed = pd.read_csv(file_path, sep=r"\s+")
    # Convert to GHz
    sed["GHz"] = sed["um"].apply(
        lambda x: (x * U.um).to(U.GHz, equivalencies=U.spectral()).value
    )
    # Re normalize the SED and convert to Jy from erg/s/Hz
    sed, lum_infrared_erg_s, lum_infrared = normalize_sed(
        sed, lum_infrared, solid_angle, cont_sens, freq_min, freq_max, remote
    )
    #  Flux (Jy) =L (erg/s/Hz) * 10^23 /  * 4 pi d^2(cm)
    flux_infrared = lum_infrared_erg_s * 1e23 / solid_angle  # Jy * Hz
    # flux_infrared_jy = flux_infrared  / (sed['GHz'].values *
    # U.GHz).to(U.Hz).value  # Jy
    sed.drop(columns=["um", "erg/s/Hz"], inplace=True)
    sed = sed.sort_values(by="GHz", ascending=True)
    return sed, flux_infrared, lum_infrared

In [15]:
main_path = os.path.dirname(os.getcwd())
sim_output_dir = os.path.join(main_path, 'experimental')
rest_freq, line_names = uas.get_line_info(main_path)
metadata = pd.read_csv(os.path.join(main_path, 'metadata','qso_metadata.csv'))
metadata = metadata.iloc[0]
freq_support = metadata['Freq.sup.']
source_freq = metadata['Freq'] * U.GHz
cont_sens = metadata['Cont_sens_mJybeam']
antenna_array = metadata['antenna_arrays']
band_range, central_freq, t_channels, delta_freq = freq_supp_extractor(
            freq_support, source_freq
        )
channel_size = band_range / t_channels

ual.generate_antenna_config_file_from_antenna_array(
            antenna_array, main_path, sim_output_dir
        )
antennalist = os.path.join(sim_output_dir, "antenna.cfg")
max_baseline = (
            ual.get_max_baseline_from_antenna_config(None, antennalist)
            * U.km)
beam_size = ual.estimate_alma_beam_size(
            central_freq, max_baseline, return_value=False
        )
snr = 1
beam_solid_angle = np.pi * (beam_size / 2) ** 2
cont_sens = cont_sens * U.mJy / (U.arcsec**2)
cont_sens_jy = (cont_sens * beam_solid_angle).to(U.Jy)
cont_sens = cont_sens_jy * snr
n_channels = t_channels
redshift = 0
#n = 1 #number of lines
type_ = 'point'
db_line = uas.read_line_emission_csv(
            os.path.join(main_path, "brightnes", "calibrated_lines.csv"),
            sep=",",
        )
central_freq = central_freq.value
band_range = band_range.value
source_freq = source_freq.value
cont_sent = cont_sens.value
freq_min = central_freq - band_range / 2
freq_max = central_freq + band_range / 2
print(freq_min, freq_max, band_range)
sed, flux_infrared, lum_infrared = sed_reading(
            type_,
            os.path.join(main_path, "brightnes"),
            cont_sens.value,
            freq_min,
            freq_max,
            False,
            None,
            None
        )

  

268.76 270.63 1.8700000000000045


In [17]:
print(f'The source frequency range and band are : {freq_min, freq_max, band_range}')
min_delta_v = 300 
max_delta_v = 450
c_km_s = c.to(U.km / U.s)

#line_names = ['H24α 669.87', 'HCN(5-4) 676.71', 'HCO+(5–4) 672.33']
line_names = None
n_lines = 10
redshift = None

if line_names is None:
    if n_lines is not None:
        n = n_lines
    else:
        n = 1
else:
    n = len(line_names)

if line_names is not None:
    db_line = db_line[db_line['Line'].isin(line_names)]

db_line['redshift'] = (db_line['freq(GHz)'].values - source_freq ) / source_freq
db_line = db_line.loc[~((db_line['redshift'] < 0) | (db_line['redshift'] > 20))]
delta_v = np.random.uniform(min_delta_v, max_delta_v, len(db_line)) * U.km / U.s
db_line["shifted_freq(GHz)"] = db_line["freq(GHz)"] / (1 + db_line['redshift'])
fwhms = (
        0.84
        * (db_line["shifted_freq(GHz)"].values * (delta_v / c_km_s) * 1e9)
        * U.Hz
        )
fwhms_GHz = fwhms.to(U.GHz).value
db_line["fwhm_GHz"] = fwhms_GHz
found_lines = 0
i = 0
lines_fitted, lines_fitted_redshifts = [], []
if redshift is not None:
    db_line['redshift_distance'] = np.abs(db_line['redshift'] - redshift)
    db_line = db_line.sort_values(by='redshift_distance')
for i in range(len(db_line)):
    db = db_line.copy()
    first_line = db.iloc[i]
    db["shifted_freq(GHz)"] = db["freq(GHz)"] / (1 + first_line['redshift'])
    db['distance(GHz)'] = abs(db['shifted_freq(GHz)'] - first_line['shifted_freq(GHz)'])
    compatible_lines = db.loc[db['distance(GHz)'] < band_range]
    compatible_lines.loc[:, 'redshift'] = np.ones(len(compatible_lines)) * first_line['redshift']
    found_lines = len(compatible_lines)
    lines_fitted.append(found_lines)
    lines_fitted_redshifts.append(first_line['redshift'])
    i += 1
if redshift is None:
    found_lines = np.max(lines_fitted)
else:
    found_lines = lines_fitted[np.argmin(np.abs(np.array(lines_fitted_redshifts) - redshift))]
if found_lines < n:
    if redshift is None:
        i = np.argmax(lines_fitted)
    else:
        i = np.argmin(np.abs(np.array(lines_fitted_redshifts) - redshift))
        print('OOOOOH', i, lines_fitted[i], lines_fitted_redshifts[i])
    first_line = db_line.iloc[i]
    db_line["shifted_freq(GHz)"] = db_line["freq(GHz)"] / (1 + first_line['redshift'])
    db_line['distance(GHz)'] = abs(db_line['shifted_freq(GHz)'] - first_line['shifted_freq(GHz)'])
    compatible_lines = db_line.loc[db_line['distance(GHz)'] < band_range]
    compatible_lines['redshift'] = first_line['redshift']
    found_lines = len(compatible_lines)
    if found_lines > 1:
        mean, std = np.mean(compatible_lines['freq(GHz)']), np.std(compatible_lines['freq(GHz)'])
    else:
        mean = np.mean(compatible_lines['freq(GHz)'])
        std =  np.random.uniform(0.1, 0.3) * band_range

    freqs = np.array(list(np.random.normal(mean, std, n - found_lines)))
    mean, std = np.mean(compatible_lines['c']), np.std(compatible_lines['c'])
    cs = np.array(list(np.random.normal(mean, std, n - found_lines)))
    mean, std = np.mean(compatible_lines['err_c']), np.std(compatible_lines['err_c'])
    err_cs = np.array(list(np.random.normal(mean, std, n - found_lines)))
    line_names = np.array([f'fake_line {i}' for i in range(n - found_lines)])
    redshifts = np.array(list(np.ones(len(freqs)) * first_line['redshift']))
    shifted_freqs = np.array(freqs / (1 + first_line['redshift']))
    distances = np.array(abs(shifted_freqs - first_line['shifted_freq(GHz)']))
    fwhms = np.array(list(np.ones(len(line_names)) * first_line['fwhm_GHz'].astype(float)))
    if redshift is None:
         data = np.column_stack((line_names, np.round(freqs, 2).astype(float), 
                                np.round(cs, 2).astype(float), np.round(err_cs, 2).astype(float), 
                                np.round(redshifts, 6).astype(float), np.round(shifted_freqs, 6).astype(float), 
                                np.round(fwhms, 6).astype(float), np.round(distances, 6).astype(float)))
    else:
        redshift_distance = np.array(list(np.ones(len(line_names)) * first_line['redshift_distance'].astype(float)))
        data = np.column_stack((line_names, np.round(freqs, 2).astype(float), 
                                np.round(cs, 2).astype(float), np.round(err_cs, 2).astype(float), 
                                np.round(redshifts, 6).astype(float), np.round(shifted_freqs, 6).astype(float), 
                                np.round(fwhms, 6).astype(float), redshift_distance, np.round(distances, 6).astype(float))
                                )
    fake_db = pd.DataFrame(data=data, columns=db_line.columns)
    for col in fake_db.columns[1:]:
        fake_db[col] = pd.to_numeric(fake_db[col])
    compatible_lines = pd.concat((compatible_lines, fake_db), ignore_index=True,)
compatible_lines = compatible_lines.reset_index(drop=True)
for index, row in compatible_lines.iterrows():
    lower_bound, upper_bound = row['shifted_freq(GHz)'] - row['fwhm_GHz']/ 2, row['shifted_freq(GHz)'] + row['fwhm_GHz'] / 2
    while lower_bound < freq_min and upper_bound > freq_max:
        row['fwhm_GHz'] -= 0.1
        lower_bound = row['shifted_freq(GHz)'] - row['fwhm_GHz'].astype(float) / 2
        upper_bound = row['shifted_freq(GHz)'] + row['fwhm_GHz'].astype(float) / 2
    if row['fwhm_GHz'] != compatible_lines['fwhm_GHz'].iloc[index]:
        compatible_lines['fwhm_GHz'].iloc[i] = row['fwhm_GHz']
print(compatible_lines)





The source frequency range and band are : (np.float64(268.76), np.float64(270.63), np.float64(1.8700000000000045))
               Line  freq(GHz)     c  err_c  redshift  shifted_freq(GHz)  \
0       H24α 669.87     447.54 -7.34   1.52  0.653345         270.687628   
1   HCN(5-4) 676.71     443.02 -5.22   0.31  0.653345         267.953776   
2  HCO+(5–4) 672.33     445.90 -6.77   0.41  0.653345         269.695700   
3   LiH(1–0) 675.28     443.95 -7.47   2.10  0.653345         268.516272   
4            OH 446     446.29 -6.55   0.24  0.653345         269.931585   
5       fake_line 0     445.95 -6.73   1.01  0.653345         269.726115   
6       fake_line 1     445.68 -6.84   1.21  0.653345         269.560094   
7       fake_line 2     446.68 -7.28   0.61  0.653345         270.164603   
8       fake_line 3     444.59 -7.16  -0.10  0.653345         268.900786   
9       fake_line 4     446.99 -6.13   0.53  0.653345         270.355323   

   fwhm_GHz  distance(GHz)  
0  0.286205       0

/var/folders/66/76q50s015tq8wc5qz1pvg7y40000gn/T/ipykernel_45467/3405677814.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compatible_lines['redshift'] = first_line['redshift']
